In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [2]:
# df_test = pd.read_csv("Groceries data test.csv")
df = pd.read_csv("Groceries data train.csv")

/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'Groceries data train.csv'

In [ ]:
print("Data shape:", df.shape)
print("\nFirst few rows of the dataset:")
df.head()

In [ ]:
# checking for null or missing value
print("\nMissing values in each column:")
df_missing = pd.DataFrame((round(100 * (df.isnull().sum() / len(df.index)), 2)), columns=['missing'])
df_missing

In [ ]:
df['itemDescription'].unique()

In [ ]:
# Clean itemDescription
df['itemDescription'] = df['itemDescription'].str.strip()

In [ ]:
basket_df = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index()
transactions = basket_df['itemDescription'].tolist()

In [ ]:
basket_df.head()

In [ ]:
transactions[0]

In [ ]:
# Top 20 items
item_counts = Counter([item for sublist in transactions for item in sublist])
print(item_counts.most_common(20))

In [ ]:
# 3. Transaction Encoding for Apriori
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
df_encoded.head()

In [ ]:
# 4. Frequent Itemset Mining with Apriori
# Parameter tuning
# min_support_values = [0.01, 0.02, 0.03, 0.05]
min_support_values = [0.002,0.003,0.004, 0.005, 0.008]
results = {}

for min_support in min_support_values:
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)
    results[min_support] = frequent_itemsets
    print(f"\nFrequent itemsets for min_support={min_support}:")
    # print(frequent_itemsets.sort_values('support', ascending=False).head(20))
    print(frequent_itemsets.sort_values('support', ascending=False))


In [ ]:
# Show frequent itemsets with more than one item
for min_support in min_support_values:
    fi = results[min_support]
    multi_itemsets = fi[fi['itemsets'].apply(lambda x: len(x) > 1)]
    print(f"\nFrequent itemsets with more than one item for min_support={min_support}:")
    print(multi_itemsets)

In [ ]:

chosen_support = 0.002
frequent_itemsets = results[chosen_support]

# 5. Association Rule Mining
min_confidence_values = [0.01, 0.05, 0.08, 0.09, 0.1]
for min_conf in min_confidence_values:
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_conf)
    rules = rules.sort_values('lift', ascending=False)
    print(f"\nAssociation Rules for min_confidence={min_conf}:")
    print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(20))

# Choose a min_confidence for visualization (e.g., 0.2)
chosen_conf = 0.08
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=chosen_conf)

In [ ]:
frequent_itemsets = apriori(df_encoded, min_support=0.003, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.08)
rules = rules.sort_values(['confidence','lift'],ascending=[False,False])
print(rules)

Using min_support = 0.003 and min_confidence = 0.1, we identified 62 association rules. The strongest rule, (frankfurter) → (other vegetables), has a confidence of 10.7% and a lift of 1.15, indicating that customers who buy frankfurters are more likely to also buy other vegetables. Most rules involve “whole milk” or “other vegetables”, reflecting their popularity. While most lifts are modest, these rules provide actionable insights for product placement and cross-selling.

Support: Most rules have support just above 0.003 (0.3%), meaning they occur in at least ~81 baskets out of 27,000. This is reasonable for actionable but not too rare patterns.

Confidence: All rules have confidence above 10% (your chosen threshold), meaning the consequent appears at least 10% of the time when the antecedent is present.

Lift: Most lift values are between 0.9 and 1.15. A lift > 1 means the rule is better than random; only a few rules (notably (frankfurter) → (other vegetables)) exceed 1.

Other metrics (leverage, conviction, etc.): Most are close to zero, indicating weak but not spurious associations.

In [ ]:
# Add additional metrics (e.g., coverage, coverage ratio)
rules['coverage'] = rules['antecedents'].apply(
    lambda x: frequent_itemsets[frequent_itemsets['itemsets'] == x]['support'].values[0]
)
rules['zhang_metric'] = (rules['confidence'] - rules['consequent support']) / (1 - rules['consequent support'])

# --------------------------------------------
# 2. Display Top Rules by Different Metrics
# --------------------------------------------
def display_top_rules(rules, metric='lift', top_n=5):
    return rules.sort_values(metric, ascending=False).head(top_n)

print("\nTop 5 Rules by Lift:")
display(display_top_rules(rules, 'lift'))

print("\nTop 5 Rules by Confidence:")
display(display_top_rules(rules, 'confidence'))

print("\nTop 5 Rules by Support:")
display(display_top_rules(rules, 'support'))

# --------------------------------------------
# 3. Visualize Rule Quality
# --------------------------------------------
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='confidence', 
    y='lift', 
    size='support', 
    hue='conviction', 
    data=rules,
    alpha=0.7,
    palette='viridis'
)
plt.title('Association Rules: Lift vs Confidence')
plt.xlabel('Confidence')
plt.ylabel('Lift')
plt.axhline(1, linestyle='--', color='red')  # Lift=1 baseline
plt.show()

# --------------------------------------------
# 4. Filter and Interpret Actionable Rules
# --------------------------------------------
# Filter rules with lift
actionable_rules = rules.sort_values('lift', ascending=False)

print("\nActionable Rules:")
display(actionable_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

In [ ]:
# Top N frequent itemsets
top_n = 10
top_itemsets = frequent_itemsets.sort_values('support', ascending=False).head(top_n)
plt.figure(figsize=(10,6))
sns.barplot(x='support', y=top_itemsets['itemsets'].astype(str), data=top_itemsets)
plt.title(f'Top {top_n} Frequent Itemsets (min_support={chosen_support})')
plt.xlabel('Support')
plt.ylabel('Itemset')
plt.show()

# Support vs Confidence Scatter Plot
plt.figure(figsize=(8,6))
sns.scatterplot(x='support', y='confidence', size='lift', data=rules, legend=False, alpha=0.7)
plt.title('Support vs Confidence of Association Rules')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.show()

# Lift Distribution Histogram
plt.figure(figsize=(8,6))
sns.histplot(rules['lift'], bins=20, kde=True)
plt.title('Distribution of Lift Values')
plt.xlabel('Lift')
plt.ylabel('Frequency')
# plt.show()

In [ ]:
def recommend_items(items, rules, top_n=5):
    items_set = set(items)
    recommendations = {}

    for idx, row in rules.iterrows():
        antecedent = set(row['antecedents'])
        consequent = set(row['consequents'])

        # If all antecedent items are in the basket and none of the consequent items are
        if antecedent.issubset(items_set) and not consequent.issubset(items_set):
            for item in consequent:
                if item not in items_set:
                    # Keep the highest confidence for each recommended item
                    if item not in recommendations or recommendations[item] < row['confidence']:
                        recommendations[item] = row['confidence']

    # Sort recommendations by confidence, descending
    recommended_list = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return recommended_list[:top_n]

# Example usage:
# Suppose rules is your DataFrame of association rules
# and the basket contains 'frankfurter' and 'sausage'
basket = ['frankfurter', 'sausage']
top_recs = recommend_items(basket, rules, top_n=3)
print("Recommendations:", top_recs)


In [ ]:
basket = ['whole milk']
top_recs = recommend_items(basket, rules, top_n=3)
print("Recommendations:", top_recs)

In [23]:
def save_rules_to_csv(rules, filename='association_rules.csv'):
    # Convert frozensets in 'antecedents' and 'consequents' to strings for CSV readability
    rules_to_save = rules.copy()
    if 'antecedents' in rules_to_save.columns:
        rules_to_save['antecedents'] = rules_to_save['antecedents'].apply(lambda x: ', '.join(list(x)))
    if 'consequents' in rules_to_save.columns:
        rules_to_save['consequents'] = rules_to_save['consequents'].apply(lambda x: ', '.join(list(x)))

    rules_to_save.to_csv(filename, index=False)
    print(f"Rules saved to {filename}")


save_rules_to_csv(rules)


/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


NameError: name 'rules' is not defined